In [1]:
import sys
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import toml
import time
from ipywidgets import *
import cv2
import uuid
from absl import logging
logging.set_verbosity(0)

import copy
import os
import json

from datetime import datetime

In [13]:
dataset=r"/atidata/demo/comstar/2021-09-21-08-53-57-mini20-03-manual_comstar_mapping_run/"
map_dir=r""


In [14]:
class Submap:
    def __init__(self, pose, grid_res, grid_alpha):
        self.pose = pose.copy()
        self.grid_alpha = grid_alpha
        self.grid_res = grid_res
        self.grid = Grid2D(self.grid_alpha, self.grid_res )
        self.num_insertions = 0
        self.finished = False
        self.node_ids = []
        self.frame_ids = []
        self.local_insertion_poses = None
    
    def search(self, frame, search_space, count_once = True):
        return self.grid.search(frame, search_space, count_once=count_once)
    
    def insert_points(self, frame, pose, frame_id, node_id):        
        if not self.finished:
            self.num_insertions +=1
            self.grid.insert_points(pose, frame)
            self.node_ids.append(node_id)
            self.frame_ids.append(frame_id)
            if self.local_insertion_poses is None:
                self.local_insertion_poses = pose.reshape(1,3)
            else:
                self.local_insertion_poses = np.concatenate((self.local_insertion_poses, pose.reshape(1,3)), axis = 0)
        else:
            print("Submap update is finished. Not inserting ")

In [15]:
def get_2D_rotation_mat(theta):
    cth = np.cos(theta)
    sth = np.sin(theta)
    return np.array([[cth, -sth],
                    [sth, cth]])

def get_angle_from_2D_rotation_mat(R):
    return np.arctan2(R[1,0], R[0,0])

def get_yelli_rotation(theta):
    th = theta - np.pi/2
    R = get_2D_rotation_mat(th)
    return R

def get_inverse_yelli_rotation(theta):
    return get_yelli_rotation(theta).T

def get_yelli_transform(pose):
    R = get_yelli_rotation(pose[2])
    t = pose[:2]
    return R,t

def get_inverse_yelli_transform(pose):
    R = get_inverse_yelli_rotation(pose[2])
    t = -R @ pose[:2].reshape(2,1)
    return R,t.flatten()

def normalize_pose(theta):
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    return theta

def get_inverse_yelli_pose(pose):
    #theta = -pose[2] + np.pi
    #theta = normalize_pose(theta)
    R,t = get_inverse_yelli_transform(pose) 
    theta = get_angle_from_2D_rotation_mat(R) + np.pi/2 
    theta = normalize_pose(theta)
    return np.array([t[0], t[1], theta])



def combine_yelli_poses(p1, p2):
    R1,t1 = get_yelli_transform(p1)
    R2,t2 = get_yelli_transform(p2)
    R = R1 @ R2
    t = (t1.reshape(2,1) + R1 @ t2.reshape(2,1)).flatten()
    theta = get_angle_from_2D_rotation_mat(R) + np.pi/2
    theta = normalize_pose(theta)
    return np.array([t[0], t[1], theta])
    
def imu_submap_pose_estimate(imu_tracker, time, submap_pose, pose):
    spose = combine_yelli_poses(pose, get_inverse_yelli_pose(submap_pose))
    dquat = imu_tracker.get_gyro_quaternion(time)
    dtheta = qe.get_roll_pitch_yaw_from_quaternion(dquat)[-1]
    pose_estimate = np.copy(spose)
    pose_estimate[2] += dtheta
    return pose_estimate

In [16]:
max_frames = 32000 
data_dir = dataset
imu_data = load_data(data_dir)
ts = np.mean(np.diff(imu_data.time))
dt = imu_ts[np.argmin(np.abs(imu_ts - ts))]
#imu_tracker = ImuTracker(dt)
imu_tracker = ImuTracker()
imu_bias_initialized = False
i = 0
print(f"")
while not imu_bias_initialized:
    imu_bias_initialized = imu_tracker.get_initial_gyro_bias(imu_data.iloc[i])
    i += 1
best_scores, poses, frame_list = [], [], []
submaps = []

z_slices = [(zmin,zmax)]

print("Using Z slices:", z_slices)

#pose = np.array([[0, 0, np.pi / 2 + tilt]] * len(z_slices))
pose = np.array([0.,0., np.pi/2])
prev_pose = pose.copy()

active_submaps = []
finished_submaps = []

spose = pose
#spose = np.array([1,2,0])
submap = Submap(spose.copy(), grid_res, grid_alpha)
active_submaps.append(submap)

node_poses = []
insertion_fr_ids=[]

i, frame_id = 0, -1
debug_data_dir = data_dir
bootstrap_frames =20
try:
    lpb = lidar_pb.LidarSmallPb(data_dir)
except:
    lpb = lidar_pb.LidarPb(data_dir)
prev_frame_time, prev_frame = lpb.get_frame(start_frame - 2)

# if use_16_beams:
#     if prev_frame.shape[0] == 32768:
#         print("data has 16 beams only")
#         use_16_beams = False
#     else:
#         print("using 16 beams only")

num_frames = np.minimum(len(lpb.list_frames()), max_frames)
print(f"total frames {num_frames}")
filtered_pts = []
for frame_id in range(start_frame, num_frames - 1):
    #if frame_id % 2 != 0:
        #continue
    try:
        frame_time, frame = lpb.get_frame(frame_id)
    except:
        print(f"skipping {frame_id}. divide-by-zero error")
        continue

    frame = np.copy(frame[:, :6]).astype(np.float64)
#     if use_16_beams:
#         ##### fix for 32 beams
#         frame = frame.reshape(-1,2048,6)
#         if frame.shape[0] == 32:
#             # select any 16 beams
#             beams = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31]
#             frame = frame[beams]
#         frame = frame.reshape(-1,6)
#         ##### fix for 32 beams

#     if correct_roll_pitch:
#         frame, roll, pitch = lidar_utils.correct_tilt(
#             frame, roll, pitch, lidar_ht=lidar_ht, use_cuda=use_cuda
#         )
#         if abs(roll) > 0.05 or abs(pitch) > 0.05:
#             print(f"Warning: Roll {roll:.2f}, Pitch  {pitch:0.2f}")

    frame = frame[(frame[:,4]>0) & (frame[:,4] < 60)]
    frame = frame[(frame[:,2] > zmin) & (frame[:,2] < zmax)]
    
    #filtered_frame = voxel_filter(frame, voxel_size=0.05)
    filtered_frame,_ = voxel_filter_fast(voxel_filter(frame, voxel_size=0.05), trunc=False, voxel = 0.5)
    #filtered_frame_z = multimap.split_frame(filtered_frame)
    
    frame_z = voxel_filter(frame[:,:3], voxel_size=0.05)

    
    #filtered_pts.append(filtered_frame_z[0].shape[0])

    if frame_id < start_frame + bootstrap_frames:
        #multimap.insert_points(filtered_frame_z, pose[0])
        #print("pose", pose, frame_z.shape)
        print("Num Bootstrap insertions", active_submaps[0].num_insertions)
        #ANUJ: local submap pose
        spose = combine_yelli_poses(pose, get_inverse_yelli_pose(active_submaps[0].pose))
        node_poses.append(pose) #ANUJ: Node is every inserted pose
        insertion_fr_ids.append(frame_id)
        active_submaps[0].insert_points(frame_z, spose, frame_id, node_id = len(node_poses)-1 )
        frame_list.append(frame_id)
        poses.append(pose)
        prev_insert_ts = frame_time
        prev_frame_time, prev_frame = frame_time, frame
        last_inserted_pose = pose.copy()
        continue

    relevant_imu_data = imu_data[imu_data["time"].between(prev_frame_time, frame_time)]
    for imu_idx in range(relevant_imu_data.shape[0]):
        imu_tracker.add_imu_data(relevant_imu_data.iloc[imu_idx])

    pose_estimate = imu_pose_estimate(imu_tracker, frame_time, pose)
    spose_estimate = combine_yelli_poses(get_inverse_yelli_pose(active_submaps[0].pose),pose_estimate)
    #spose_estimate = imu_submap_pose_estimate(imu_tracker, frame_time, active_submaps[0].pose, pose)
    #print(frame_id,pose_estimate)
    search_space = utils.grid_space(center=spose_estimate, **grid_params)
    #scores = multimap.search(filtered_frame_z, search_space)
    #print()
    scores = active_submaps[0].search(filtered_frame, search_space, count_once = False)
    #print(scores)
    best = np.argmax(scores)
    #pose = [search_space[b] for b in best]
    #best_scores.append([scores[i][b] for i, b in enumerate(best)])
    best_scores.append(scores[best])
    #pose = select_best_pose(pose, best_scores[-1])
    
    spose = search_space[best]
    pose = combine_yelli_poses(active_submaps[0].pose, spose)
    
    
    #print(pose)
    #if (np.linalg.norm(pose[:2]-prev_pose[:2])> 0.1 or (prev_insert_ts - frame_time)>1) :
    if (np.linalg.norm(pose[:2]-last_inserted_pose[:2])> 0.1 or (prev_insert_ts - frame_time)>1) :
        #multimap.insert_points(filtered_frame_z, pose[0])
        #multimap.insert_points(frame_z, pose[0])
        
        #print("Num insertions", active_submaps[0].num_insertions)
        #print("Fr id", frame_id)
        #print("sub pose", active_submaps[0].pose)
        node_poses.append(pose)
        insertion_fr_ids.append(frame_id)
        active_submaps[0].insert_points(frame_z, spose, frame_id, node_id=len(node_poses)-1)
        if len(active_submaps) ==1:
            
            if active_submaps[0].num_insertions >= int(num_submap_range_data/2):
                active_submaps.append(Submap(pose.copy(), grid_res, grid_alpha))
                s2pose = combine_yelli_poses(get_inverse_yelli_pose(active_submaps[1].pose),pose)
                active_submaps[1].insert_points(frame_z,s2pose, frame_id, node_id=len(node_poses)-1)
                #print(f"spose {spose}, s2pose {s2pose}")
                
        else:
            
            s2pose = combine_yelli_poses(get_inverse_yelli_pose(active_submaps[1].pose),pose)
            active_submaps[1].insert_points(frame_z,s2pose, frame_id, node_id=len(node_poses)-1)
            #print(f"len else spose {spose}, s2pose {s2pose}")
            
        if active_submaps[0].num_insertions >= num_submap_range_data - 1:
            active_submaps[0].finished = True
            finished_submaps.append(active_submaps.pop(0))
            active_submaps.append(Submap(pose.copy(), grid_res, grid_alpha))
            s2pose = combine_yelli_poses(get_inverse_yelli_pose(active_submaps[1].pose), pose)
            active_submaps[1].insert_points(frame_z,s2pose, frame_id, node_id=len(node_poses)-1)
        
        #print("num active submaps", len(active_submaps))
        prev_insert_ts = frame_time
        last_inserted_pose = pose.copy()
    #multimap.insert_points(filtered_frame_z, pose[0])
     
    prev_pose = pose.copy()
    frame_list.append(frame_id)    
    poses.append(pose)
    if frame_id % 250 == 0:
        print(
            f"frame:{frame_id} scores:{best_scores[-1]} best pose:{pose}  "
        )
    prev_frame_time, prev_frame = frame_time, frame

#multimap.transform_scores()
poses = np.array(poses)
node_poses = np.array(node_poses)
insertion_fr_ids = np.array(insertion_fr_ids)

print(f"final pose {poses[-1]}")
#print(f"Min points in frame: {np.min(filtered_pts)}")
# if pruned:
#     print("Pruning the map..")
#     grid_x_limit, grid_y_limit = get_prune_limits(multimap.maps[0], poses)
#     multimap = prune_map(multimap, grid_x_limit, grid_y_limit)
# if not save_as_float:
#     multimap.maps[0].grid = (multimap.maps[0].grid * 128).astype(np.uint8)
#     multimap.maps[0].origin.dtype = "float"
# multimap.save_map(map_name)
# np.save(f"{map_name}/yelli_poses.npy", poses)
# np.save(f"{map_name}/yelli_frames.npy", frame_list)
# with open(os.path.join(map_name, "map_metadata.json"), "w") as f:
#     json.dump(generate_map_metadata(data_dir), f)
# quality = map_quality(multimap.maps[0].grid)
# print(f"map quality {quality}")
# alpha_arr = [1.0, 0.7]



NameError: name 'load_data' is not defined